In [ ]:
#about torch...
import torch
import torch.nn as nn
import torch
import torch.optim as optim
import torch.nn.functional as F

from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Dataset

#using numpy
import numpy as np

#for data load or save
import pandas as pd

#visualize some datasets
import matplotlib.pyplot as plt

#check our work directory
import os



lr = 0.001 # learning_rate
batch_size = 100 # we will use mini-batch method
epochs = 200 # How much to train a model
train = True

# os.listdir('data/train')

device = 'cuda' 

torch.manual_seed(1234)
if device == 'cuda':
    torch.manual_seed(1234)
    # torch.mps.manual_seed_all(1234)
    # os.makedirs('data/catORdog', exist_ok=True)
    base_dir = 'archive'
    train_dir = 'archive/seg_train'
    test_dir = 'archive/seg_pred'
    train_list = []
    test_list = []
    from pathlib import Path
    for path in Path(train_dir).rglob('*.jpg'):
        train_list.append(path)
    for path in Path(test_dir).rglob('*.jpg'):
        test_list.append(path)

    from PIL import Image

    random_idx = np.random.randint(1, 12000, size=10)


    from sklearn.model_selection import train_test_split
    train_list, val_list = train_test_split(train_list, test_size=0.2)

    # data Augumentation
    train_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ])

    val_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ])

    test_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ])


    class dataset(torch.utils.data.Dataset):

        def __init__(self, file_list, transform=None):
            self.file_list = file_list
            self.transform = transform

        # dataset length
        def __len__(self):
            self.filelength = len(self.file_list)
            return self.filelength

        # load an one of images
        def __getitem__(self, idx):
            img_path = self.file_list[idx]
            img = Image.open(img_path)
            img_transformed = self.transform(img)
            
            label = img_path.parts
            if img_path.parts[1] == 'seg_train':
                if label[2] == 'kirill':
                    label = 0
                elif label[2] == 'oleg':
                    label = 1
                elif label[2] == 'vlad':
                    label = 2
                elif label[2] == 'roma':
                    label = 3
               
            else:
                
                label = label[2].split('.')[0]+'.'+label[2].split('.')[1]
            return img_transformed, label


    train_data = dataset(train_list, transform=train_transforms)
    test_data = dataset(test_list, transform=test_transforms)
    val_data = dataset(val_list, transform=test_transforms)
    
    train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(dataset=val_data, batch_size=batch_size, shuffle=True)


    class Cnn(nn.Module):
        def __init__(self):
            super(Cnn, self).__init__()

            self.layer1 = nn.Sequential(
                nn.Conv2d(3, 16, kernel_size=3, padding=0, stride=2),
                nn.BatchNorm2d(16),
                nn.ReLU(),
                nn.MaxPool2d(2)
            )

            self.layer2 = nn.Sequential(
                nn.Conv2d(16, 32, kernel_size=3, padding=0, stride=2),
                nn.BatchNorm2d(32),
                nn.ReLU(),
                nn.MaxPool2d(2)
            )

            self.layer3 = nn.Sequential(
                nn.Conv2d(32, 64, kernel_size=3, padding=0, stride=2),
                nn.BatchNorm2d(64),
                nn.ReLU(),
                nn.MaxPool2d(2)
            )

            self.fc1 = nn.Linear(3 * 3 * 64, 254)
            self.dropout = nn.Dropout(0.5)
            self.fc2 = nn.Linear(254, 120)
            self.fc3 = nn.Linear(120, 80)
            self.fc4 = nn.Linear(80, 40)
            self.fc5 = nn.Linear(40, 4)
            
            

        def forward(self, x):
            out = self.layer1(x)
            out = self.layer2(out)
            out = self.layer3(out)
            out = out.view(out.size(0), -1)
            out = self.fc1(out)
            out = self.fc2(out)
            out = self.fc3(out)
            out = self.fc4(out)
            out = self.fc5(out)
            return out


    model = Cnn().to(device)
    if os.path.isfile("CNNmodel.pt"):
        model.load_state_dict(torch.load('CNNmodel.pt'))
    model.train()

    optimizer = optim.Adam(params=model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    
    lastAccuracy = 0
    nowAccuracy = 0
    if train == True :
        print("Train Starts")
        for epoch in range(epochs):
            print("Epoch Starts")
            epoch_loss = 0
            epoch_accuracy = 0
        

            # if os.path.isfile("CNNmodel.pt"):
            #     model.load_state_dict(torch.load('CNNmodel.pt'))
            for data, label in train_loader:
                data = data.to(device)
                
                label = label.to(device)
                

                output = model(data)
                
                loss = criterion(output, label)

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                acc = ((output.argmax(dim=1) == label).float().mean())
                epoch_accuracy += acc / len(train_loader)
                epoch_loss += loss / len(train_loader)

            print('Epoch : {}, train accuracy : {}, train loss : {}'.format(epoch + 1, epoch_accuracy, epoch_loss))

            with torch.no_grad():
                epoch_val_accuracy = 0
                epoch_val_loss = 0
                for data, label in val_loader:
                    data = data.to(device)
                
                    label = label.to(device)

                    val_output = model(data)
                    val_loss = criterion(val_output, label)

                    acc = ((val_output.argmax(dim=1) == label).float().mean())
                    epoch_val_accuracy += acc / len(val_loader)
                    epoch_val_loss += val_loss / len(val_loader)
                nowAccuracy = 100 * epoch_val_accuracy   
                print('Epoch : {}, val_accuracy : {}, val_loss : {}'.format(epoch + 1, epoch_val_accuracy, epoch_val_loss))
                print(f'Accuracy of the network on the 10000 test images: {100 * epoch_val_accuracy} %')
            if(nowAccuracy > lastAccuracy):
                torch.save(model.state_dict(), 'CNNmodel.pt')
                print("Progress Saved")
                lastAccuracy = nowAccuracy
    

    if train == False:
        class_ = {0: 'kirill', 1: 'oleg', 2: 'vlad', 3: 'roma'}
        results = []
        model.eval()
        with torch.no_grad():
            for data, fileid in test_loader:
                data = data.to(device)
                preds = model(data)
                
                _, preds = torch.max(preds, dim=1)
                
                results += list(zip(list(fileid), preds.tolist()))
                

       
       
       
        idx = list(map(lambda x: x[0], results))
        prob = list(map(lambda x: x[1], results))

        submission = pd.DataFrame({'id': idx, 'label': prob})
        
        

        import random

        id_list = []
        
        
        fig, axes = plt.subplots(2, 5, figsize=(5, 5), facecolor='w')
       

        for ax in axes.ravel():

            i = random.choice(submission['id'].values)
           
            label = submission.loc[submission['id'] == i, 'label'].values[0]
            
            

            img_path = os.path.join(test_dir, '{}.jpg'.format(i))
            img = Image.open(img_path)
            img = img.resize((224, 224))
            ax.set_title(class_[label])
            ax.axis('off')
            ax.imshow(img)
           
        
        plt.show()
    
       
        


Train Starts
Epoch Starts
Epoch : 1, train accuracy : 0.47805556654930115, train loss : 1.2247724533081055
Epoch : 1, val_accuracy : 0.5994117856025696, val_loss : 1.0529590845108032
Accuracy of the network on the 10000 test images: 59.94117736816406 %
Progress Saved
Epoch Starts
Epoch : 2, train accuracy : 0.5464583039283752, train loss : 1.0777558088302612
Epoch : 2, val_accuracy : 0.6682353019714355, val_loss : 0.9708294868469238
Accuracy of the network on the 10000 test images: 66.82353210449219 %
Progress Saved
Epoch Starts
Epoch : 3, train accuracy : 0.5850000381469727, train loss : 0.9715657234191895
Epoch : 3, val_accuracy : 0.619411826133728, val_loss : 0.8208858966827393
Accuracy of the network on the 10000 test images: 61.94118118286133 %
Epoch Starts
Epoch : 4, train accuracy : 0.6094443798065186, train loss : 0.9153999090194702
Epoch : 4, val_accuracy : 0.6201961040496826, val_loss : 0.8576059341430664
Accuracy of the network on the 10000 test images: 62.01961135864258 %
E

Epoch : 35, train accuracy : 0.8184027671813965, train loss : 0.45894116163253784
Epoch : 35, val_accuracy : 0.8149019479751587, val_loss : 0.5117992758750916
Accuracy of the network on the 10000 test images: 81.49019622802734 %
Epoch Starts
Epoch : 36, train accuracy : 0.7804861068725586, train loss : 0.538226306438446
Epoch : 36, val_accuracy : 0.7978430986404419, val_loss : 0.4646163284778595
Accuracy of the network on the 10000 test images: 79.78430938720703 %
Epoch Starts
Epoch : 37, train accuracy : 0.7961804866790771, train loss : 0.4957093298435211
Epoch : 37, val_accuracy : 0.8078430891036987, val_loss : 0.4187275171279907
Accuracy of the network on the 10000 test images: 80.78430938720703 %
Epoch Starts
Epoch : 38, train accuracy : 0.8020139336585999, train loss : 0.48182961344718933
Epoch : 38, val_accuracy : 0.824509859085083, val_loss : 0.5470873117446899
Accuracy of the network on the 10000 test images: 82.45098876953125 %
Epoch Starts
Epoch : 39, train accuracy : 0.83881